### Baseline (Source-only with .train mode)

In [1]:
# import modules and libraries
import torch
import torch.nn as nn
from torch.optim import Adam

# functions to create datasets and models
from data.data import *
from models.models import *

In [2]:
# check if CUDA is available
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available')
else:
    print('CUDA is available')
    device = torch.device('cuda')

CUDA is available


### Parameters

In [3]:
"""Parameters for this experiment"""
# params for loading the data
path = 'data/Paderborn_FD/'
source_domain = 'a'
target_domains = ['b', 'c', 'd']

# params for configuring the model
input_dim = 1
hidden_dim = 256
output_dim = 3
drop_prob = 0.5

# params for training the model
epochs_pre = 50 # pre-training
epochs = 50

# optimizing the model
lr = 1e-4
batch_size = 20
beta1 = 0.5
beta2 = 0.9

### Data Loading

In [4]:
# source domain data for training, validating and testing
train_dataloader_src, val_dataloader_src, test_dataloader_src = generate_dataloaders(path, source_domain, batch_size)

# target domain data for validating and testing
train_dataloader_tgt, val_dataloader_tgt, test_dataloader_tgt = [], [], []
for target_domain in target_domains:
    train_dataloader_temp, val_dataloader_temp, test_dataloader_temp = generate_dataloaders(path, target_domain, batch_size)
    
    train_dataloader_tgt.append(train_dataloader_temp)
    val_dataloader_tgt.append(val_dataloader_temp)
    test_dataloader_tgt.append(test_dataloader_temp)

### Model

In [5]:
encoder_src = Encoder_AMDA(
    input_dim=input_dim, # 1D input (sensor readings)
    hidden_dim=hidden_dim # hidden layers 64
)

classifier = Classifier_AMDA(
    hidden_dim=hidden_dim, # hidden layers 64
    output_dim=output_dim, # 3 classes (healthy, inner- and outer-bearing damages)
    dropout=drop_prob # dropout prob 0.5
)

encoder_tgt = Encoder_AMDA(
    input_dim=input_dim, # 1D input (sensor readings)
    hidden_dim=hidden_dim, # hidden layers 64
)

if train_on_gpu:
    encoder_src = encoder_src.to(device)
    classifier = classifier.to(device)
    encoder_tgt = encoder_tgt.to(device)

#### Functions for training and testing the model

In [6]:
# functions for training source encoder and shared classifier
def train_src(encoder, classifier):
    """Train the source encoder and shared classifier on source domain"""
    encoder.train()
    classifier.train()

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(
        list(encoder.parameters()) + list(classifier.parameters()),
        lr=lr,
        betas=(beta1, beta2)
    )

    for epoch in range(epochs_pre):
        accuracies = []
        losses = []

        for batch_idx, (x_src, y_src) in enumerate(train_dataloader_src):
            if train_on_gpu:
                x_src, y_src = x_src.to(device), y_src.to(device)

            optimizer.zero_grad()

            e_src = encoder(x_src)
            pred_src = classifier(e_src)
            loss_src = criterion(pred_src, y_src)
            loss_src.backward()

            optimizer.step()

            losses.append(loss_src.detach().item()) # detach the loss from compute graph 
            accuracies.append(y_src.eq(pred_src.detach().argmax(dim=1)).float().mean())
        
        # print for each epoch
        print(f'Epoch: {epoch + 1} \n \t Train Loss:{torch.tensor(losses).mean():0.2f} \t Train Acc:{torch.tensor(accuracies).mean():0.2f}')

        # for each epoch, evaluate your model on the validation data
        val_loss, val_acc = evaluate_src(encoder, classifier, phase = 'Val')
        print(f'\t Val Loss:{val_loss:0.2f} \t\t Val Acc:{val_acc:0.2f}')

    return encoder, classifier

# testing model on source domain
def evaluate_src(encoder, classifier, phase='Val'): # phase can be validate or test
    """Evaluate the trained network on source domain"""
    encoder.eval()
    classifier.eval()

    criterion = nn.CrossEntropyLoss()
    
    losses = []
    accuracies = []
    dataloader = val_dataloader_src if phase == 'Val' else test_dataloader_src
    
    with torch.no_grad():
        for batch_idx, (x_src, y_src) in enumerate(dataloader):
            if train_on_gpu:
                x_src, y_src = x_src.to(device), y_src.to(device)

            e_src = encoder(x_src)
            pred_src = classifier(e_src)
            loss_src = criterion(pred_src, y_src)
            
            # append loss for each batch
            losses.append(loss_src.detach().item()) # detach the loss from compute graph 
            accuracies.append(y_src.eq(pred_src.detach().argmax(dim=1)).float().mean())
            
        mean_loss = torch.tensor(losses).mean()
        mean_acc = torch.tensor(accuracies).mean()
        return mean_loss, mean_acc

In [7]:
# testing model on target domains
def evaluate_tgt(encoder, classifier, dataloader):
    """Evaluate the network on current target domain"""
    encoder.train()
    classifier.train()

    criterion = nn.CrossEntropyLoss()
    
    losses = []
    accuracies = []
    
    with torch.no_grad():
        for batch_idx, (x, y) in enumerate(dataloader): # target dataloader
            if(train_on_gpu):
                x, y = x.to(device), y.to(device)

            pred_tgt = classifier(encoder(x))
            loss_tgt = criterion(pred_tgt, y)
            
            # append loss for each batch
            losses.append(loss_tgt.detach().item()) # detach the loss from compute graph 
            accuracies.append(y.eq(pred_tgt.detach().argmax(dim=1)).float().mean())
            
        mean_loss = torch.tensor(losses).mean()
        mean_acc = torch.tensor(accuracies).mean()
        return mean_loss, mean_acc

In [8]:
# pre-training
encoder_src, classifier = train_src(encoder_src, classifier)

Epoch: 1 
 	 Train Loss:0.81 	 Train Acc:0.56
	 Val Loss:0.53 		 Val Acc:0.75
Epoch: 2 
 	 Train Loss:0.46 	 Train Acc:0.80
	 Val Loss:0.44 		 Val Acc:0.80
Epoch: 3 
 	 Train Loss:0.41 	 Train Acc:0.81
	 Val Loss:0.40 		 Val Acc:0.81
Epoch: 4 
 	 Train Loss:0.33 	 Train Acc:0.83
	 Val Loss:0.33 		 Val Acc:0.82
Epoch: 5 
 	 Train Loss:0.21 	 Train Acc:0.91
	 Val Loss:0.36 		 Val Acc:0.82
Epoch: 6 
 	 Train Loss:0.16 	 Train Acc:0.93
	 Val Loss:0.34 		 Val Acc:0.84
Epoch: 7 
 	 Train Loss:0.12 	 Train Acc:0.95
	 Val Loss:0.29 		 Val Acc:0.87
Epoch: 8 
 	 Train Loss:0.09 	 Train Acc:0.97
	 Val Loss:0.21 		 Val Acc:0.90
Epoch: 9 
 	 Train Loss:0.07 	 Train Acc:0.97
	 Val Loss:0.15 		 Val Acc:0.93
Epoch: 10 
 	 Train Loss:0.06 	 Train Acc:0.98
	 Val Loss:0.12 		 Val Acc:0.95
Epoch: 11 
 	 Train Loss:0.06 	 Train Acc:0.98
	 Val Loss:0.10 		 Val Acc:0.96
Epoch: 12 
 	 Train Loss:0.05 	 Train Acc:0.98
	 Val Loss:0.08 		 Val Acc:0.97
Epoch: 13 
 	 Train Loss:0.05 	 Train Acc:0.98
	 Val Loss:0.0

### Continuous Unsupervised Adaptation

In [10]:
encoder_tgt.load_state_dict(encoder_src.state_dict())

for phase in range(len(target_domains)):
    if(phase == 0): # Domain A to B
        print(f'\nPhase {phase + 1} (Domain {source_domain} to Domain {target_domains[phase]})')
    else: # Domain B to C, and Domain C to D
        print(f'\nPhase {phase + 1} (Domain {target_domains[phase - 1]} to Domain {target_domains[phase]})')

    print('\nTesting Accuracy on Previously Seen and Current Domains:')
    print(f'Source Domain {source_domain}:')
    src_test_loss, src_test_acc = evaluate_src(encoder_tgt, classifier, phase='Test')
    print(f'\t Test (with Source) Loss:{src_test_loss:0.2f} \t Test (with Source) Acc:{src_test_acc:0.2f}') 
    for p in range(phase + 1):
        print(f'Target Domain {target_domains[p]}:')
        tgt_test_loss, tgt_test_acc = evaluate_tgt(encoder_tgt, classifier, test_dataloader_tgt[phase])
        print(f'\t Test (with Target) Loss:{tgt_test_loss:0.2f} \t Test (with Target) Acc:{tgt_test_acc:0.2f}')


Phase 1 (Domain a to Domain b)

Testing Accuracy on Previously Seen and Current Domains:
Source Domain a:
	 Test (with Source) Loss:0.03 	 Test (with Source) Acc:0.99
Target Domain b:
	 Test (with Target) Loss:24.67 	 Test (with Target) Acc:0.34

Phase 2 (Domain b to Domain c)

Testing Accuracy on Previously Seen and Current Domains:
Source Domain a:
	 Test (with Source) Loss:0.03 	 Test (with Source) Acc:0.99
Target Domain b:
	 Test (with Target) Loss:16.74 	 Test (with Target) Acc:0.47
Target Domain c:
	 Test (with Target) Loss:16.66 	 Test (with Target) Acc:0.46

Phase 3 (Domain c to Domain d)

Testing Accuracy on Previously Seen and Current Domains:
Source Domain a:
	 Test (with Source) Loss:0.03 	 Test (with Source) Acc:0.99
Target Domain b:
	 Test (with Target) Loss:20.38 	 Test (with Target) Acc:0.34
Target Domain c:
	 Test (with Target) Loss:20.40 	 Test (with Target) Acc:0.34
Target Domain d:
	 Test (with Target) Loss:20.79 	 Test (with Target) Acc:0.34
